## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

- 12 колонок, 21525 строчек
- в столбцах days_employed и total_income есть пропуски
- в столбцах family_status, gender, income_type и purpose необходимо обратить внимание на тип данных
- в столбцах days_employed и total_income лучше заменить тип данных на int64 для большей наглядности, но специфика данных в столбце days_employed не позволяет говорить о корректности находящихся там данных

### Шаг 2. Предобработка данных

### Обработка пропусков

Посмотрим на количество пропусков в столбцах days_employed и total_income

In [3]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

По 2174 пропуска в этих столбцах. Взглянем на них подробнее.
Для удобства фильтрации временно заполним эти позиции нулями.

In [4]:
df = df.fillna(0)

Проверим.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Теперь посмотрим есть ли на самом деле у этих заемщиков работа.

In [7]:
df[df['total_income']==0]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Как видно у каждого заемщика есть работа. Судя по всему сюда закралась ошибка и необходимо ее исправить, заполнив характерными значениями.
Так как для ответа на поставленные вопросы нам необходим только столбец total_income, то оставим столбец days_employed.
Чтобы определить требуемое значение вместо нуля необходимо сначала понять какой метод подсчета характерного значения нам требуется.
Для этого посмотрим на выборку уникальных значений по всей таблице по каждому из нужных нам типов income_type.

In [8]:
df[df['income_type']=='сотрудник']['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

Значений много, они разные, попробуем посмотреть на максимум. Минимум будет равен нулю.

In [9]:
df[df['income_type']=='сотрудник']['total_income'].max()

1726276.0143316737

1726276 - многовато для сотрудника. Поэтому для определния характерного значения возьмем метод медианы.

In [10]:
df[df['income_type']=='сотрудник']['total_income'].median()

133546.4572377553

133546 - характерное значение прибыли для сотрудника. Сделаем то же самое для остальных категорий заемщиков.

In [11]:
df[df['income_type']=='пенсионер']['total_income'].max()

735103.2701666558

735103 - для пенсионера чересчур. Берем медиану.

In [12]:
df[df['income_type']=='пенсионер']['total_income'].median()

110179.69076140685

110179 - характерное значение прибыли для пенсионера.

In [13]:
df[df['income_type']=='компаньон']['total_income'].max()

2265604.028722744

2265603 - пожалуй также медиана.

In [14]:
df[df['income_type']=='компаньон']['total_income'].median()

162401.35155456822

162401 - характерное значение прибыли для компаньона.

In [15]:
df[df['income_type']=='госслужащий']['total_income'].max()

910451.4708141336

910451 - медиана точно.

In [16]:
df[df['income_type']=='госслужащий']['total_income'].median()

139034.45238630794

139034 - характерное значение прибыли для госслужащего.

In [17]:
df[df['income_type']=='предприниматель']['total_income'].max()

499163.1449470857

499163 - также возьмем медиану.

In [18]:
df[df['income_type']=='предприниматель']['total_income'].median()

249581.57247354285

249581 - характерное значение прибыли для госслужащего.

Теперь необходимо заменить нули нужными нам данными.
Напишем функцию для этого.

In [19]:
def income_not_null(row):
    null = row['total_income']
    i_type = row['income_type']

    if null == 0: 
        if i_type == 'сотрудник':
            return 133546
        elif i_type == 'компаньон':
            return 162401
        elif i_type == 'пенсионер':
            return 110719
        elif i_type == 'госслужащий':
            return 139034
        elif i_type == 'предприниматель':
            return 249581
    else: 
        return null

Применим получившуюся функцию.    

In [20]:
df['total_income'] = df.apply(income_not_null, axis=1)

Проверим себя.

In [21]:
df[df['total_income']==0]['income_type'].value_counts()

Series([], Name: income_type, dtype: int64)

Нулевых значений в столбце total_income не осталось, результат достигнут.

### Вывод

В стобцах days_employed и total_income было одинаковое количество пропущенных значений. Казалось бы что у этих заемщиков никогда не было работы и не было никакого дохода. Но при более подробном рассмотрении оказалось, что это не так. Так как количество пропусков составляло около 10% от общего количества заемщиков, то их потеря могла бы оказаться губительной для итоговой статистики. Было принято решение по заполнению пропусков в графе total_income(единственной актуальной для нас для ответа на поставленные вопросы) путем вычисления медианы по категориям сотрудников, к которым относятся заемщики с пропущенной информацией. Среднее арифмитическое по этому столбцу нам не подходит, потому что есть аномально большие значения. Таким образом, заполнив пропуски характерным значением прибыли для этих заемщиков, мы не только убрали все пропуски, но и сможем дать более обоснованные ответы на поставленные вопросы.
Можем рекомендовать заказчику поставить фильтр при заполнении данной информации - не пропускать анкету без показателя дохода заемщика(строго положительное число). Также можно рекомендовать фильтр на показатель продолжительности трудоустройства - строго положительный, исчисляется днями.

### Замена типа данных

In [22]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Стобцы days_employed и total_income переведны в int64 для большей наглядности при анализе. Замена остальных столбцов с типом object не требуется, либо по причине наличия дублирующего столбца-идентификатора (_id), либо по причине логической(purpose) - для наглядности при аналитике лучше оставить такой тип данных.

### Обработка дубликатов

Узнаем есть ли дублирующиеся строчки

In [23]:
df.duplicated().sum()

54

Удалим их методом drop_duplicates со сбросом ненужных индексов(reset_index(drop=True))

In [24]:
df.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21467,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21469,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


Проверим значения в столбце education.

In [25]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Данные подходящие, но требуется привести все к нижнему регистру.

In [26]:
df['education'] = df['education'].str.lower()

Проверим значения

In [27]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Цель достигнута.
Проверим данные в столбце children.

In [28]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Аномальными выглядят значения "-1" и "20", посмотрим сколько их.

In [29]:
df[df['children'] < 0]['children'].value_counts()

-1    47
Name: children, dtype: int64

С отрицательным значением 47 человек. Скорее всего это человеческий фактор и лишнее тире перед значением.
Заменим на положительные значения.

In [30]:
df['children'] = df['children'].abs()

Проверим.

In [31]:
df['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Теперь посмотрим на многодетных.

In [32]:
df[df['children'] == 20]['children'].value_counts()

20    76
Name: children, dtype: int64

Целых 76 человек заявляют о ровно 20 детях, ни больше, ни меньше. Взгялнем на них подробнее.

In [33]:
print(df[df['children'] == 20])

       children  days_employed  dob_years education  education_id  \
606          20           -880         21   среднее             1   
720          20           -855         44   среднее             1   
1074         20          -3310         56   среднее             1   
2510         20          -2714         59    высшее             0   
2941         20          -2161          0   среднее             1   
...         ...            ...        ...       ...           ...   
21008        20          -1240         40   среднее             1   
21325        20           -601         37   среднее             1   
21390        20              0         53   среднее             1   
21404        20           -494         52   среднее             1   
21491        20           -173         27   среднее             1   

         family_status  family_status_id gender income_type  debt  \
606    женат / замужем                 0      M   компаньон     0   
720    женат / замужем           

Безусловно, может быть все что угодно, но выборка заемщиков с 20 детьми не повзоялет думать, что это правдивые значения.
Скорее всего и сюда закрался человеческий фактор, ведь на клавиатурах на numpad 2 и 0 очень близко.
Если отрицательное количество детей могло повлиять на ответы на поставленные здесь вопросы, то разница между 20 и 2 несущественна.
Поэтому замену делать не будем, но обратим внимание заказчика на этот факт.

Проверим значения в столбце dob_years.

In [34]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Оказалось, что есть еще неродившиеся. Посмотрим сколько их.

In [35]:
df[df['dob_years'] == 0]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

101 человек. Взглянем на них.

In [36]:
print(df[df['dob_years'] == 0])

       children  days_employed  dob_years education  education_id  \
99            0         346541          0   среднее             1   
149           0          -2664          0   среднее             1   
270           3          -1872          0   среднее             1   
578           0         397856          0   среднее             1   
1040          0          -1158          0    высшее             0   
...         ...            ...        ...       ...           ...   
19829         0              0          0   среднее             1   
20462         0         338734          0   среднее             1   
20577         0         331741          0   среднее             1   
21179         2           -108          0    высшее             0   
21313         0          -1268          0   среднее             1   

               family_status  family_status_id gender income_type  debt  \
99           женат / замужем                 0      F   пенсионер     0   
149                в 

Опять-таки нет никаких критериев в этой выборке, чтобы определить системность этой ошибки, поэтому можно опять сослаться на
человеческий фактор - просто забыли внести эту информацию в анкету. Учитывая, что возраст нас не интересует при ответах на 
поставленные вопросы, то просто оставим как есть, но передадим заказчику рекомендацию поставить фильтр на ввод этого параметра в 
анкете - не меньше 18, не больше 99, не отрицательные числа. 

### Вывод

В таблице было 54 дублирующихся строки. Найдены благодаря оператору duplicated и удалены при помощи drop_duplicates  со сбросом ненужных индексов(reset_index(drop=True)). Вероятно внесены туда ошибочно - человеческий фактор, двойное нажатие. Их удаление не повлияет на итоговый отчет, так как составляет всего 0,25% от всей информации, а также облегчит таблицу, объем и увеличит быстродействие.
Также проверили три столбца на предмет ошибок аномалий. Отрицательное количество детей заменили на положительное. Заменили регистр в стобце образования. Обнаружили 101 запись с нулевым значением в стобце dob_years.
Рекомендуем заказчику поставить фильтры на введение следующих значений в анкету:
1. В столбце 'children' значения больше или равны нулю.
2. Во всех столбцах не с количественной информацией использовать только нижний регистр.
3. В столбце 'dob_years' положительные значения от 18 до 99.

### Лемматизация

Вызовем необходимый нам инструмент лемматизации.

In [37]:
m = Mystem()

Обозначим функцию для получения нового столбца с леммами.

Прогоним этот столбец через Counter.

In [39]:
from collections import Counter
lemmas_list=[]
for row in df['purpose']:
    lemmas_list.extend(m.lemmatize(row))
    
Counter(lemmas_list).most_common(20)

[(' ', 33677),
 ('\n', 21525),
 ('недвижимость', 6367),
 ('покупка', 5912),
 ('жилье', 4473),
 ('автомобиль', 4315),
 ('образование', 4022),
 ('с', 2924),
 ('операция', 2610),
 ('свадьба', 2348),
 ('свой', 2235),
 ('на', 2233),
 ('строительство', 1881),
 ('высокий', 1375),
 ('получение', 1316),
 ('коммерческий', 1315),
 ('для', 1294),
 ('жилой', 1233),
 ('сделка', 944),
 ('дополнительный', 909)]

### Вывод

Основными причинами кредитования являются покупка автомобиля, покупка недвижимости, образование и свадебные мероприятия.

### Категоризация данных

Проведем категоризацию для удобства анализа назначения кредитования. 
Присвоим всем значениям стобца purpose четыре категории, полученные при лемматизации.

In [40]:
def strict_p(row):
    lemmas = m.lemmatize(row)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas:
        return 'потребительский'
    elif 'ремонт' in lemmas:
        return 'потребительский'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'недвижимость'


df['strict_purpose'] = df['purpose'].apply(strict_p)

Проверим.

In [41]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,strict_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,потребительский
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Вывод

Подготовили таблицу для дальнейшего анализа. Теперь можно приступать к ответам на поставленные вопросы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос необходимо сопоставить данные из столбца children и столбца debt.
Посчиатем заемщиков по количеству детей. 

In [42]:
df.pivot_table(index='children',values='debt',aggfunc='count')

,debt
children,
0,14149
1,4865
2,2055
3,330
4,41
5,9
20,76


Создадим таблицу с этими данными child_1.

In [43]:
child_1 = df.pivot_table(index='children',values='debt',aggfunc='count')

Просуммируем количество должников по количеству детей.

In [44]:
df.pivot_table(index='children',values='debt',aggfunc='sum')

,debt
children,
0,1063
1,445
2,194
3,27
4,4
5,0
20,8


Создадим таблицу с этими данными child_2.

In [45]:
child_2 = df.pivot_table(index='children',values='debt',aggfunc='sum')

Разделим одну таблицу на другую для получения процента просрочек по категориям заемщиков и назовем новую таблицу child_total.

In [46]:
child_total = child_2['debt'] / child_1['debt']*100

Выведем на экран информацию по должникам

In [47]:
print(child_total)

children
0      7.512898
1      9.146968
2      9.440389
3      8.181818
4      9.756098
5      0.000000
20    10.526316
Name: debt, dtype: float64


### Вывод

Согласно этим цифрам получается, что отсутствие детей у заемщика положительно влияет на его имидж для кредитного комитета. Отсутствие просрочек у заемщика с пятью детьми - аномалия, вписывающаяся в общую теорию. Всего заемщиков с пятью детьми 9, а средний показатель просрочек для заемщиков с детьми около 10%. В эту выборку не попал десятый заемщик, который возможно дал бы необходимый показатель.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос необходимо сопоставить данные из столбца debt и столбца family_status_id.
Посчиатем количество заемщиков по семейному положению. 

In [48]:
df.pivot_table(index='family_status',values='debt',aggfunc='count')

,debt
family_status,
Не женат / не замужем,2813
в разводе,1195
вдовец / вдова,960
гражданский брак,4177
женат / замужем,12380


Создадим таблицу с этими данными family_1.

In [49]:
family_1 = df.pivot_table(index='family_status',values='debt',aggfunc='count')

Просуммируем количество должников по количеству детей.

In [50]:
df.pivot_table(index='family_status',values='debt',aggfunc='sum')

,debt
family_status,
Не женат / не замужем,274
в разводе,85
вдовец / вдова,63
гражданский брак,388
женат / замужем,931


Создадим таблицу с этими данными family_2.

In [51]:
family_2 = df.pivot_table(index='family_status',values='debt',aggfunc='sum')

Разделим одну таблицу на другую для получения процента просрочек по категориям заемщиков и назовем новую таблицу family_total.

In [52]:
family_total = family_2['debt'] / family_1['debt']*100

Выведем на экран информацию по должникам.

In [53]:
print(family_total)

family_status
Не женат / не замужем    9.740491
в разводе                7.112971
вдовец / вдова           6.562500
гражданский брак         9.288963
женат / замужем          7.520194
Name: debt, dtype: float64


### Вывод

Вывод первый: если смотреть на процентное соотношение, то большой зависимости между семейным положением и возвратом кредита в срок не существует. Все категории попали в диапазон от 7,11% до 9,74% вероятности просрочки кредита.
Вывод второй: если требуется обозначить более приоритетную категорию заемщика по семейному положению, то в лидеры вырываюся вдовцы, сразу за ними заемщики в разводе, а замыкают ТОП-3 заемщики, живущие в законном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос необходимо изучить столбец total_income и поделить его на несколько категорий по уровню доходов.
Определим пять категорий: "низкий доход" - ниже 30000, "низкий средний доход" - от 30000 до 60000, 
"средний доход" - от 60000 до 120000, "высокий средний" - от 120000 до 250000 и "высокий" - больше 250000.
Создадим новый столбец total_income_category для разделения всех заемщиков. Для этого создадим функцию income_category.

In [54]:
def income_category(row):
    total_income = row
    if total_income <= 30000:
        return 'низкий доход'
    elif total_income > 30000 and total_income <= 60000:
        return 'средний низкий доход'
    elif total_income > 60000 and total_income <= 120000:
        return 'средний доход'
    elif total_income > 120000 and total_income <= 250000:
        return 'средний высокий доход'
    else:
        return 'высокий доход'
df['total_income_category'] = df['total_income'].apply(income_category)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,strict_purpose,total_income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний высокий доход
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,потребительский,средний высокий доход
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий доход
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,средний высокий доход
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний высокий доход
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,потребительский,средний доход
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний высокий доход


Проверим.

In [55]:
df['total_income_category'].value_counts()

средний высокий доход    11455
средний доход             6452
высокий доход             2812
средний низкий доход       784
низкий доход                22
Name: total_income_category, dtype: int64

Теперь необходимо сопоставить данные из столбца debt и столбца total_income_category.
Посчиатем количество заемщиков по семейному положению. 

In [56]:
df.pivot_table(index='total_income_category',values='debt',aggfunc='count')

,debt
total_income_category,
высокий доход,2812
низкий доход,22
средний высокий доход,11455
средний доход,6452
средний низкий доход,784


Создадим таблицу с этими данными family_1.

In [57]:
total_income_category_1 = df.pivot_table(index='total_income_category',values='debt',aggfunc='count')

Просуммируем количество должников по количеству детей.

In [58]:
df.pivot_table(index='total_income_category',values='debt',aggfunc='sum')

,debt
total_income_category,
высокий доход,194
низкий доход,2
средний высокий доход,961
средний доход,537
средний низкий доход,47


Создадим таблицу с этими данными family_2.

In [59]:
total_income_category_2 = df.pivot_table(index='total_income_category',values='debt',aggfunc='sum')

Разделим одну таблицу на другую для получения процента просрочек по категориям заемщиков и назовем новую таблицу final_income_category.

In [60]:
final_income_category = total_income_category_2['debt'] / total_income_category_1['debt']*100

Выведем на экран информацию по должникам.

In [61]:
print(final_income_category)

total_income_category
высокий доход            6.899004
низкий доход             9.090909
средний высокий доход    8.389350
средний доход            8.323001
средний низкий доход     5.994898
Name: debt, dtype: float64


### Вывод

Большой зависимости между уровнем дохода заемщика и его способностью возвращать кредит в срок не наблюдается. Погрешность между категориями примерно такая же как и в предыдущих вопросах. Лидер - заемщик с доходом от 30000 до 60000, второе место занял заемщик с доходом более 250000, третье место у заемщиков с доходом ниже 30000.
Казалось бы что наименее благоприятными для выдачи займа будут клиенты с низким доходом, но аналитика это не подтверждает. 
Было бы неплохо добавить в подсчеты суммы займа и суммы ежемесячного платежа, а также соотнести их с фактическим уровнем дохода каждого конкретного заемщика, чтобы получить определенный процентный барьер ниже которого резко вырастает вероятность просрочки по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос необходимо сопоставить данные из столбца debt и столбца strict_purpose.
Посчиатем количество заемщиков по типу займа. 

In [62]:
df.pivot_table(index='strict_purpose',values='debt',aggfunc='count')

,debt
strict_purpose,
автомобиль,4315
недвижимость,10228
образование,4022
потребительский,2960


Создадим таблицу с этими данными purpose_1.

In [63]:
purpose_1 = df.pivot_table(index='strict_purpose',values='debt',aggfunc='count')

Просуммируем количество должников по количеству детей.

In [64]:
df.pivot_table(index='strict_purpose',values='debt',aggfunc='sum')

,debt
strict_purpose,
автомобиль,403
недвижимость,747
образование,370
потребительский,221


Создадим таблицу с этими данными purpose_2.

In [65]:
purpose_2 = df.pivot_table(index='strict_purpose',values='debt',aggfunc='sum')

Разделим одну таблицу на другую для получения процента просрочек по категориям заемщиков и назовем новую таблицу purpose_total.

In [66]:
purpose_total = purpose_2['debt'] / purpose_1['debt']*100

Выведем на экран информацию по должникам.

In [67]:
print(purpose_total)

strict_purpose
автомобиль         9.339513
недвижимость       7.303481
образование        9.199403
потребительский    7.466216
Name: debt, dtype: float64


### Вывод

Также как и предыдущих вопросах большого влияния на вероятность возврата кредита в срок цель кредита не дает. 
Лидер по количеству возвращенных в срок займов - недвижимость, что вероятно связано с боязнью за свой дом и с тем что условия кредитования в этом случае намного жестче и потенциальный заемщик оценивается в разы пристальнее.
Второе место у потребительских кредитов. Суммы этих кредитов обычно небольшие, поэтому их возвращать проще и охотнее.
Третье место у образования. Думаю, что данный кредит берут только те учащиеся, которые очень сильно рассчитывают на дорогостоящую учебу, способную положительно повлиять на их дальнейшую жизнь, поэтому они более ответственны и строже следят за выплатами.
Четвертое место у автокредитов. Получить их несложно, можно найти субсидируемые, возвратность почти на уровне образовательных кредитов. Автомобиль у населения зачастую это показатель статуса, а боязнь этого лишится не повзоляет людям просрочить выплаты.
Также как и в предыдущем вопросе и во всех остальных хочется сравнить эти показатели с суммой кредитования и суммой ежемесячных выплат.

### Шаг 4. Общий вывод

Каждый из вопросов в отдельности не дал полновесного ответа какой из заемщиков надежнее. Но если собрать все воедино, то ТОП-3 идеальных кейсов будут следующие:

1. Вдовец/Вдова(93.44) без детей(92.19) с доходом от 30000 до 60000(94.01) с кредитом на покупку недвижимости(92.7). (93,085 процент вероятности возврата кредита в срок.

2. Разведенный(92,89) без детей(92.19) с доходом от 30000 до 60000(94.01) с кредитом на покупку недвижимости(92.7). 92.9475 процент вероятности возврата кредита в срок.

3. Вдовец/Вдова(93.44) без детей(92.19) с доходом более 250000(93.1) с кредитом на покупку недвижимости(92.7). 92,8575 процент вероятности возврата кредита в срок.

Также необходимо отметить худший кейс:

1. Не женат / не замужем(90.26) с детьми(90.77) с доходом от 120000 до 250000(91.48) с кредитом на автомобиль(90.66). 90,7925 процент вероятности возврата кредита в срок.

Эти метрики не являются основными при скоринге заемщика и могут иметь вес только при пограничном решении по итогам оценки первостепенных показателей. Логично, что имея кого-либо в паре или на иждевении, заемщик будет стараться погашать кредит в срок, но, опять-таки, нельзя это рассматривать как основные показатели - слишком невелика разница как между различными категорями внутри одного вопроса, так и сравнивая лучший и худший итоговые кейсы - менее 4% вероятности.










### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.